In [38]:
# setup cell!!

from canvasapi import Canvas
from canvasapi.exceptions import BadRequest
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
import ipywidgets as widgets
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd

# setup canvas connection
tokenfile = open("canvas-token", "r")
token = tokenfile.readline()
tokenfile.close()
url = "https://westminster.instructure.com/"
canvas = Canvas(url, token)

# now i'ma hardcode the courses that people will ask for extensions in
# this will avoid a number of (slow!) requests and filters.
math202id = 3387585
math202 = canvas.get_course(math202id)
phys309id = 3388078
phys309 = canvas.get_course(phys309id)
coursedict = {
    "MATH 202": math202,
    "PHYS 309": phys309
}

# setup google connection
# If modifying these scopes, delete the file token.json.
scopes = ['https://www.googleapis.com/auth/spreadsheets.readonly']
spreadsheet_id = '1mtc172TQIju8BwAiqwFS8Fwtz1I8ZN6Aw7SwfrVvdG0'
data_range = 'B2:I' #this probably works for now

creds = None
# The file google-token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first time.
if os.path.exists('google-token.json'):
    creds = Credentials.from_authorized_user_file('google-token.json', scopes)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'google-credentials.json', scopes)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('google-token.json', 'w') as token:
        token.write(creds.to_json())


In [8]:
# read data from spreadsheet
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=data_range).execute()
    requests = result.get('values', [])
    requestsdf = pd.DataFrame(requests, columns = ["name", "email", "classname", 
        "assignment", "due date", "need1", "need2", "status"])
    # pull out the ones that I need to do something with
    # tododf = requestsdf[pd.isna(requestsdf["status"])]
    tododf = requestsdf

    if not values:
        print('No data found.')
except HttpError as err:
    print(err)

In [2]:
def build_override_dict(inlist):
    [name, email, coursename, assignment, dd, need1, need2, status] = inlist

    # find the correct course and lookup the student
    course = coursedict[coursename]
    try: student = course.get_users(search_term = email)[0]
    except IndexError as err:
        print("haha that email's wrong")
        return
    
    # parse the date correctly as a datetime object
    # comes in from google forms as, e.g., "1/30"
    duetime = timedelta(hours=23,minutes=59)
    duedate = datetime.strptime(dd+"/23", '%m/%d/%y').replace(tzinfo=ZoneInfo("America/Denver"))
    duedate = duedate + duetime

    return {
        'student_ids': [student.id],
        'due_at': duedate, 
        'all_day': True,
        'all_day_date':  str(duedate.date())
    }

In [ ]:
# so now what I am going to do is something like, 
# filter the requestsdf to only have the todo ones
# then tododf.apply(build_override_dict, axis=1) 

# at some point we also need to do the assignment lookup
# and finally commit all of those overrides to canvas.

In [2]:
emailw = widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
coursew = widgets.Dropdown(
    options=[("MATH 202", math202), ("PHYS 309", phys309)],
    #value='MATH 202',
    description='Course:',
    disabled=False,
)
display(emailw)
display(coursew)

Text(value='Hello World', description='String:', placeholder='Type something')

Dropdown(description='Course:', options=(('MATH 202', Course(_requester=<canvasapi.requester.Requester object …

In [ ]:
try:
    test_assg.create_override(
        assignment_override={
            'student_ids': [12261567],
            'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
            'all_day': True,
            'all_day_date':  '2023-02-01'
        }
    )
except BadRequest:
    bad_override = next((ov for ov in test_assg.get_overrides() if 12261567 in ov.student_ids), None)
    bad_override.edit(
        assignment_override={
            'student_ids': [12261567],
            'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
            'all_day': True,
            'all_day_date':  '2023-02-01'
        }
    )


In [ ]:
test_assg.create_override(
    assignment_override={
        'student_ids': [12273191],
        'due_at': datetime(2023,2,1,23,59,59,tzinfo=ZoneInfo('America/Denver')),
        'all_day': True,
        'all_day_date':  '2023-02-01'
    }
)

In [21]:
math202.get_users(search_term="sb0513@westminstercollege.edu")[0]

IndexError: list index out of range